<a href="https://colab.research.google.com/github/aneladavis/Fun_Recommender_System/blob/main/NBAplayerrecommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Importing and Exploration

In [ ]:
import pandas as pd

# read in the player file

df_p = pd.read_csv('all_seasons.csv',error_bad_lines=False, low_memory=False, encoding='latin-1',sep=',')

df_p.head()

In [ ]:
# import college team data
df_s = pd.read_csv('cbb21.csv',error_bad_lines=False, low_memory=False, encoding='latin-1',sep=',')
df_s.head()

# Cleaning and Condensing

In [ ]:
# function used from Natassha Selvaraj tutorial (just changed name)
def lower_entry(entry):
  result = str(entry).lower()
  return(result)


In [ ]:
# remove columns that aren't used (can change if desired) and lower the case of columns
df_p = df_p.drop(['Unnamed: 0', 'team_abbreviation', 'player_height', 'player_weight', 'country', 'draft_year', 'pts', 'reb', 'ast', 'oreb_pct', 'dreb_pct', 'usg_pct', 'ts_pct', 'ast_pct', 'season'], axis=1)

# lower text
df_p['player_name'] = df_p['player_name'].apply(lower_entry)
df_p['college'] = df_p['college'].apply(lower_entry)
df_s['TEAM'] = df_s['TEAM'].apply(lower_entry)

# replace numbering with player name
df_p.set_index('player_name', inplace=True)
df_p.head()

In [ ]:
def remove_words(entry):
  input = str(entry)
  if (input.isnumeric()):
    return entry
  else:
    return -1


In [ ]:
# remove undrafted from draft round/number column
df_p['draft_round'] = df_p['draft_round'].apply(remove_words)
df_p['draft_number'] = df_p['draft_number'].apply(remove_words)

In [ ]:
# check format
df_p.head()
# df_s.info()


In [ ]:
# add school column to main data file
empty_col = pd.DataFrame(['']*len(df_p))
df_p.insert(len(list(df_p.columns)), 'c_rank', empty_col)

In [ ]:
# function to insert team rating
def insert_barthag(entry):
  input = str(entry)
  # print(input)
  if input in list(df_s['TEAM']):
    x = list(df_s['TEAM']).index(input)
    return df_s['BARTHAG'][x]
  else:
    return -1

In [ ]:
# replace old column and drop old college column
df_p['c_rank'] = df_p['college'].apply(insert_barthag)
df_p = df_p.drop('college', axis=1)

In [ ]:
# check point
df_p.info()
df_p.index

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import StandardScaler

# data is scared differently for each entry, scale it to fit for analysis
scale = StandardScaler().fit_transform(df_p)
scale.shape

In [ ]:
# make new df
df_p_scaled = pd.DataFrame(scale,index=df_p.index, columns=df_p.columns)

In [ ]:
# check random stuff if need be


# df_p_scaled.head()
# df_p_scaled.info()
# scale.shape
# df_p_scaled.values
# df_p_scaled.head()
df_p_scaled.loc['dennis rodman',:].values[0].reshape(6,1)

In [ ]:
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

df_p_feature_matrix = csr_matrix(df_p_scaled.values)

knn_search = NearestNeighbors(metric='cosine', algorithm='brute')
knn_search.fit(df_p_feature_matrix)




player_list = []
rec_list = []

# bug here when df_p_scaled.loc[player,:].values[0].reshape(1,-1) is called on the search
for player in df_p_scaled.index:
    distances, indices = knn_search.kneighbors(df_p_scaled.loc[player,:].values[0].reshape(1,-1), n_neighbors=11)

    for elem in range(0,len(distances.flatten())):
        if elem == 0:
            player_list.append(player)
        else:
            rec_list.append([player, elem, df_p_scaled.index[indices.flatten()[elem]],distances.flatten()[elem]])


recommendation_df = pd.DataFrame(rec_list, columns=['search_player','rec_number','rec_player','distance_score'])

In [ ]:

df_p_scaled.index[0]
# df_p_scaled.loc['dennis rodman',:].values[0].reshape(6,1)

In [ ]:
recommendation_df[recommendation_df['search_player'] == 'dennis rodman']

,search_player,rec_number,rec_player,distance_score
0,dennis rodman,1,toni kukoc,0.031090
1,dennis rodman,2,rashard lewis,0.056704
2,dennis rodman,3,george hill,0.057162
3,dennis rodman,4,george hill,0.060356
4,dennis rodman,5,p.j. tucker,0.063703
5,dennis rodman,6,jeff hornacek,0.066547
6,dennis rodman,7,nate mcmillan,0.072001
7,dennis rodman,8,brian shaw,0.073769
8,dennis rodman,9,terry porter,0.074158
9,dennis rodman,10,arvydas sabonis,0.074200
